In [17]:
# install.packages("rvest")
# remotes::install_github("dmi3kno/polite")("politer")
# install.packages("xml2")
library(rvest) # rvest makes scraping easier
library(polite) # polite is the "polite" version of rvest
library(xml2) # makes it easier to work with HTML and XML from R


library(tidyverse)
library(magrittr)
library(purrr) 
library(glue) 
library(stringr)

library(dplyr)
library(lubridate)
library(writexl)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [6]:
#first we load the httr library so that we can fetch the response from an api
library(httr)
library(jsonlite)

In [35]:
# this is helper function to get the actually desired data inside the list
# this function takes 2 parameter, the first one is the data which is a list of all the information about the weather for each day, the second parameter is the index which is a string of the data we want to get
get_information_list <- function(data, index) {
    empty <- vector(mode = "list", length = 0) # first we prepare an empty list which will be appended with the information we want to get
    for(i in seq_along(data)){ # next we loop through every item inside the data given
        empty <- append(empty, data[i][[1]][[index]]) # then we append the data to the list we prepared before
    }
    return(empty)
}

#this is the function to get the weather history from a week ago until the current date
#this function will return a tibble which contains the information about the weather from 7 days ago to the current date. The information includes the date, temperature, windspeed, humidity, and conditions
#this function takes 2 parameters, first one is the city which is a string that indicates the city we want get the weather data from, next is the apiKey which is a string of the API access token from the website mentioned above
get_weather_data <- function(city, apiKey){

    currentDate <- Sys.Date() # in this code we try to
    
    lastWeek <- as.Date(currentDate) - 7 # then we also get the date from a week ago
    
    location <- c(city, city, city, city, city, city, city)
    

    response_raw <- GET(glue("https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/{lastWeek}/{currentDate}?unitGroup=metric&key={apiKey}&contentType=json")) # the we use the api to get the weather data by specifying the city and dates
    response <- content(response_raw) # here I get the body content of the response

    #then in the code below, we create new tibble/data frame and use the get_information_list() function we defined above to get the data for each column
    result_weather_data <- tibble(Location = location,
                                  Date = get_information_list(response[["days"]], "datetime"),
                                    Temperature = get_information_list(response[["days"]], "temp"),
                                    WindSpeed = get_information_list(response[["days"]], "windspeed"),
                                    Humidity = get_information_list(response[["days"]], "humidity"),
                                    Conditions = get_information_list(response[["days"]], "conditions"))

    #here we make sure each colum has correct data types
    result_weather_data <- result_weather_data %>% mutate(across(Date, as.Date)) %>%
                                                    mutate(across(Temperature, as.integer)) %>%
                                                    mutate(across(WindSpeed, as.integer)) %>%
                                                    mutate(across(Humidity, as.interger)) %>%
                                                    mutate(across(Conditions, as.character)) %>% 
                                                    mutate(across(Location, str_to_title))
    return(result_weather_data)
}

  get_weather_data_safe <- purrr::possibly(.f = get_weather_data, # so here we want to make sure that the the function is showing error message if any error happens
                                   otherwise = "Something went wrong: possibly maximum number of request has been reached")

chch_data<-get_weather_data_safe("christchurch","GMTLW3T9CX22FK8GH662Q9GPZ")
chch_data
                                                           
chch_data %<>% write.csv("chch_data.csv")
chch_data
                                                           
akl_data<-get_weather_data_safe("auckland","GMTLW3T9CX22FK8GH662Q9GPZ")
akl_data

akl_data %<>% write.csv("akl_data.csv")
akl_data
                                                          
welly_data<-get_weather_data_safe("wellington","GMTLW3T9CX22FK8GH662Q9GPZ")
welly_data
                                                           
welly_data %<>% write.csv("welly_data.csv")
welly_data

[1] "Something went wrong: possibly maximum number of request has been reached"

NULL

[1] "Something went wrong: possibly maximum number of request has been reached"

NULL

[1] "Something went wrong: possibly maximum number of request has been reached"

NULL